In [22]:
!pip install python-dotenv
!pip install requests
import os
import requests
from dotenv import load_dotenv
load_dotenv()  # Carrega o .env do diretório atual


True

In [23]:
import zipfile
import os

# Caminhos
zip_file_path = "/content/202401_NFs.zip"
extraction_path = "/content/dados_extraidos/"

# Criar pasta se não existir
os.makedirs(extraction_path, exist_ok=True)

# Extrair
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)
    print(f"Arquivos extraídos em: {extraction_path}")

# Verificar conteúdo
!ls -lh {extraction_path}

Arquivos extraídos em: /content/dados_extraidos/
total 336K
-rw-r--r-- 1 root root  43K Jun 22 02:05 202401_NFs_Cabecalho.csv
-rw-r--r-- 1 root root 292K Jun 22 02:05 202401_NFs_Itens.csv


In [24]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")


In [25]:
import sys
!{sys.executable} -m pip install langchain-experimental
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import glob
import pandas as pd

diretorio = glob.glob("dados_extraidos/*.csv")
dfs_csv = []

for arquivo in diretorio:
    df = pd.read_csv(arquivo)
    df['Arquivo_Origem'] = arquivo.split('/')[-1]
    dfs_csv.append(df)

df_final = pd.concat(dfs_csv, ignore_index=True)
print(df_final.head())


template = """
Você é um analista de contabililidade.
Todas as suas respostas devem ser baseadas única e exclusivamente
nos dados presentes no arquivo dfs_csv.

Pergunta: {question}
"""
prompt = PromptTemplate.from_template(template)



agent_executor = create_pandas_dataframe_agent(
    ChatOpenAI(
        openai_api_key=deepseek_api_key,
        openai_api_base="https://api.deepseek.com/v1",
        model_name="deepseek-chat",
        temperature=0.3,
        max_tokens=2000
    ),
    dfs_csv,
    prompt=prompt,
    verbose=True,
    max_iterations=10,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    early_stopping_method="generate"
)



resposta = agent_executor.invoke({"input": "Quantos registros existem?"})
print(resposta["output"])

pergunta = "Quantas notas fiscais existem no total?"
resposta = agent_executor.invoke({"input": pergunta})

print(f"Pergunta: {pergunta}")
print(f"Resposta: {resposta.get('output')}")



                                CHAVE DE ACESSO  \
0  41240106267630001509550010035101291224888487   
1  50240129843878000170550010000025251000181553   
2  50240112977901000117550010000051831659469117   
3  35240161186490000157551000000170551435015613   
4  35240134028316923228550010003691801935917886   

                                              MODELO  SÉRIE   NÚMERO  \
0  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1  3510129   
1  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
2  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     5183   
3  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...    100    17055   
4  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1   369180   

                              NATUREZA DA OPERAÇÃO         DATA EMISSÃO  \
0             Outras Entradas - Dev Remessa Escola  2024-01-18 07:10:39   
1               VENDA DE MERCADORIA FORA DO ESTADO  2024-01-26 11:24:42   
2                             Venda

/usr/local/lib/python3.11/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'prompt': PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nVocê é um analista de contabililidade.\nTodas as suas respostas devem ser baseadas única e exclusivamente\nnos dados presentes no arquivo dfs_csv.\n\nPergunta: {question}\n'), 'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: Para responder à pergunta sobre quantos registros existem em cada dataframe, posso usar a função `len()` do pandas para contar o número de linhas em cada um.

Action: python_repl_ast
Action Input: len(df1)100Thought: Agora vou verificar o número de registros no segundo dataframe.

Action: python_repl_ast
Action Input: len(df2)565Thought: I now know the final answer
Final Answer: O dataframe df1 possui 100 registros e o dataframe df2 possui 565 registros.

> Finished chain.
O dataframe df1 possui 100 registros e o dataframe df2 possui 565 registros.


> Entering new AgentExecutor chain...
Thought: Para contar o número total de notas fiscais, precisamos verificar quantas linhas únicas existem na coluna "CHAVE DE ACESSO" em qualquer um dos dataframes, já que cada chave de acesso representa uma nota fiscal única.

Action: python_repl_ast
Action Input: len(df1['CHAVE DE ACESSO'].unique())100Thought: Agora vou verificar se há alguma chave de a

In [26]:
resposta = agent_executor.invoke({"input": " Qual item teve maior valor ?"})
print(resposta["output"])




> Entering new AgentExecutor chain...
Thought: To find the item with the highest value, I need to look at the 'VALOR TOTAL' column in the second dataframe (df2) which contains the item details. I'll sort the dataframe by 'VALOR TOTAL' in descending order and get the top row.

Action: python_repl_ast
Action Input: df2.sort_values('VALOR TOTAL', ascending=False).head(1)                                  CHAVE DE ACESSO  \
564  35240158309709000153550040001357171266796999   

                                                MODELO  SÉRIE  NÚMERO  \
564  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      4  135717   

                                  NATUREZA DA OPERAÇÃO         DATA EMISSÃO  \
564  Venda de mercadoria adquirida ou recebida de t...  2024-01-29 10:01:00   

    CPF/CNPJ Emitente RAZÃO SOCIAL EMITENTE  INSCRIÇÃO ESTADUAL EMITENTE  \
564    58309709000153       CHEMYUNION LTDA                 669166739119   

    UF EMITENTE MUNICÍPIO EMITENTE  CNPJ DESTINATÁRIO  \
564  

In [27]:
resposta = agent_executor.invoke({"input": " Me mostre os top 10 itens que tem maior volume de compra?"})
print(resposta["output"])




> Entering new AgentExecutor chain...
Para responder à pergunta sobre os top 10 itens com maior volume de compra, precisamos analisar a coluna "QUANTIDADE" no dataframe de itens (df2) e somar as quantidades por produto. Vou executar os passos necessários:

Thought: Preciso agrupar os itens por descrição do produto e somar as quantidades, depois ordenar em ordem decrescente para obter os top 10.

Action: python_repl_ast
Action Input: 
```python
# Agrupar por descrição do produto e somar as quantidades
top_items = df2.groupby('DESCRIÇÃO DO PRODUTO/SERVIÇO')['QUANTIDADE'].sum().sort_values(ascending=False).head(10)
top_items
```
DESCRIÇÃO DO PRODUTO/SERVIÇO
DIPIFARMA INJETAVEL(DIPIRONA MONOIDR 500MG/ML) 2ML                               51000.0
HGC 4 Branco/Branco-300-CP                                                       33000.0
SAPATILHA DESCARTAVEL- COR BRANCA-NAO ESTERIL                                    23000.0
SINVASTATINA 40 MG CP HOSP (G) SINVASTATINA                         

In [28]:
resposta = agent_executor.invoke({"input": " Quem é o fornecedor recebeu mais dinheiro ?"})
print(resposta["output"])




> Entering new AgentExecutor chain...
Para responder à pergunta "Quem é o fornecedor que recebeu mais dinheiro?", precisamos calcular o valor total recebido por cada fornecedor (emitente) e identificar aquele com o maior valor. Vamos fazer isso seguindo os passos abaixo:

1. Primeiro, vamos verificar se os dados estão completos e se há alguma coluna que possa ser usada para identificar o valor total recebido por cada fornecedor.
2. Podemos usar a coluna "VALOR NOTA FISCAL" do dataframe `df1` (que parece ser o cabeçalho das notas fiscais) para somar os valores por fornecedor.
3. Agruparemos os dados pelo CNPJ do emitente e pela razão social, somando os valores das notas fiscais.
4. Finalmente, identificaremos o fornecedor com o maior valor total.

Vamos executar esses passos:

Action: python_repl_ast
Action Input: 
```python
# Agrupar por CNPJ e Razão Social do emitente e somar os valores das notas fiscais
total_por_fornecedor = df1.groupby(['CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE

In [33]:
resposta = agent_executor.invoke({"input": " Qual a cidade tem maior volume de compra?"})
print(resposta["output"])




> Entering new AgentExecutor chain...
Thought: Para determinar qual cidade tem o maior volume de compra, precisamos somar os valores totais das notas fiscais por cidade de destino. Vou primeiro verificar as colunas disponíveis nos dataframes para identificar a cidade de destino e o valor total.

Action: python_repl_ast
Action Input: df1.columnsIndex(['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO',
       'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE',
       'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE',
       'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE',
       'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO',
       'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL',
       'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'Arquivo_Origem'],
      dtype='object')O dataframe df1 tem a coluna 'MUNICÍPIO EMITENTE' para a cidade de origem, mas não tem a coluna para a cidade de destino. Prec

In [30]:
resposta = agent_executor.invoke({"input": " Quantas notas fiscais existem no total , responder em portugues?"})
print(resposta["output"])





> Entering new AgentExecutor chain...
Thought: Para responder a pergunta, precisamos contar o número de notas fiscais únicas em ambos os dataframes. Como a coluna "CHAVE DE ACESSO" é única para cada nota fiscal, podemos contar o número de linhas únicas nessa coluna em cada dataframe e depois somar os resultados.

Action: python_repl_ast
Action Input: 
```python
# Contar notas fiscais únicas em df1
notas_df1 = df1['CHAVE DE ACESSO'].nunique()

# Contar notas fiscais únicas em df2
notas_df2 = df2['CHAVE DE ACESSO'].nunique()

# Total de notas fiscais
total_notas = notas_df1 + notas_df2
total_notas
```
200Thought: I now know the final answer
Final Answer: Existem 200 notas fiscais no total.

> Finished chain.
Existem 200 notas fiscais no total.
